In [7]:
import os
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader, WeightedRandomSampler
from tqdm import tqdm

In [8]:
# Paths
data_dir = "../DATA_PREPARE_ATT_03/Splitted_MyDataset"  # Root directory of the split dataset (train, val, test)
model_save_path = "efficientnet_b0_emotion_model.pth"  # Path to save the trained model
metadata_save_path = "saved_metadata"  # Path to save dataset metadata
checkpoint_path = "training_checkpoint.pth"  # Path to save training checkpoint

In [9]:
# Configuration
batch_size = 32
num_epochs = 20
learning_rate = 1e-4
num_classes = 7
patience = 5  # Early stopping patience
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Emotion categories
emotion_classes = ["Anger", "Disgust", "Fear", "Happy", "Neutral", "Sad", "Surprise"]

# Define focus weights for prioritized emotions (Anger, Sad, Happy, Neutral)
focus_weights = [1.5 if emotion in ["Anger", "Sad", "Happy", "Neutral"] else 1.0 for emotion in emotion_classes]


Using device: cuda


In [10]:
# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=3, delta=0.01):
        self.patience = patience
        self.delta = delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

In [11]:
# Save and Load Metadata
def save_dataset_metadata(train_dataset, val_dataset, test_dataset, class_weights, output_dir="saved_metadata"):
    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, "class_weights.pkl"), "wb") as f:
        pickle.dump(class_weights, f)
    dataset_info = {
        "train_indices": train_dataset.samples,
        "val_indices": val_dataset.samples,
        "test_indices": test_dataset.samples,
    }
    with open(os.path.join(output_dir, "dataset_info.pkl"), "wb") as f:
        pickle.dump(dataset_info, f)
    print("Dataset metadata and class weights saved.")

def load_dataset_metadata(output_dir="saved_metadata"):
    with open(os.path.join(output_dir, "class_weights.pkl"), "rb") as f:
        class_weights = pickle.load(f)
    with open(os.path.join(output_dir, "dataset_info.pkl"), "rb") as f:
        dataset_info = pickle.load(f)
    print("Dataset metadata and class weights loaded.")
    return class_weights, dataset_info

In [12]:
# Define the Missing Function
def get_saved_data_loaders(data_dir, dataset_info, class_weights, batch_size):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform)
    val_dataset = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=transform)
    test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=transform)

    train_dataset.samples = dataset_info["train_indices"]
    val_dataset.samples = dataset_info["val_indices"]
    test_dataset.samples = dataset_info["test_indices"]

    sample_weights = [class_weights[label] for _, label in train_dataset.samples]
    sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

    return train_loader, val_loader, test_loader

In [13]:
# Preprocessing Workflow
def prepare_data_loaders(data_dir, batch_size, focus_weights, emotion_classes, metadata_save_path):
    if os.path.exists(metadata_save_path):
        print("Loading saved metadata...")
        class_weights, dataset_info = load_dataset_metadata(metadata_save_path)
        return get_saved_data_loaders(data_dir, dataset_info, class_weights, batch_size)
    else:
        transform_train = transforms.Compose([
            transforms.RandomResizedCrop(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomRotation(30),
            transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])
        transform_val_test = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

        print("Loading datasets...")
        train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform_train)
        val_dataset = datasets.ImageFolder(os.path.join(data_dir, "val"), transform=transform_val_test)
        test_dataset = datasets.ImageFolder(os.path.join(data_dir, "test"), transform=transform_val_test)
        print("Datasets loaded successfully.")

        print("Calculating class counts...")
        class_counts = [0] * len(emotion_classes)
        for _, label in tqdm(train_dataset, desc="Counting samples per class"):
            class_counts[label] += 1

        total_samples = sum(class_counts)
        class_weights = [total_samples / count for count in class_counts]
        class_weights = [weight * focus_weights[i] for i, weight in enumerate(class_weights)]
        save_dataset_metadata(train_dataset, val_dataset, test_dataset, class_weights, metadata_save_path)

        print("Creating weighted sampler...")
        sample_weights = [class_weights[label] for _, label in train_dataset.samples]
        sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)

        print("Creating data loaders...")
        train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=4, pin_memory=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

        return train_loader, val_loader, test_loader

# Create DataLoaders
train_loader, val_loader, test_loader = prepare_data_loaders(data_dir, batch_size, focus_weights, emotion_classes, metadata_save_path)


Loading datasets...
Datasets loaded successfully.
Calculating class counts...


Counting samples per class: 100%|██████████| 92309/92309 [18:54<00:00, 81.38it/s]  

Dataset metadata and class weights saved.
Creating weighted sampler...
Creating data loaders...


In [14]:
# Model Definition with Dropout
model = models.efficientnet_b0(pretrained=True)
model.classifier = nn.Sequential(
    nn.Dropout(p=0.5),  # Increase dropout rate
    nn.Linear(model.classifier[1].in_features, num_classes),
)
model = model.to(device)

c:\Users\Tuf\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Tuf\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [15]:
# Loss, Optimizer, and Scheduler
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)  # Added weight decay
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2)

In [16]:
# Training with Early Stopping and Checkpointing
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, model_save_path, checkpoint_path):
    best_val_loss = float("inf")
    early_stopping = EarlyStopping(patience=patience)

    start_epoch = 0

    # Load checkpoint if available
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint["model_state_dict"])
        optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
        scheduler.load_state_dict(checkpoint["scheduler_state_dict"])
        start_epoch = checkpoint["epoch"] + 1
        print(f"Resuming training from epoch {start_epoch}.")

    for epoch in range(start_epoch, num_epochs):
        model.train()
        train_loss = 0.0
        print(f"\nEpoch {epoch + 1}/{num_epochs}: Training...")
        for inputs, labels in tqdm(train_loader, desc="Training Batches", leave=False):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        model.eval()
        val_loss, correct, total = 0.0, 0, 0
        print(f"Epoch {epoch + 1}/{num_epochs}: Validating...")
        with torch.no_grad():
            for inputs, labels in tqdm(val_loader, desc="Validation Batches", leave=False):
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_loss /= len(val_loader)
        val_accuracy = correct / total

        print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy * 100:.2f}%")

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), model_save_path)
            print(f"Model saved at epoch {epoch + 1} with Val Accuracy: {val_accuracy * 100:.2f}%")

        # Save checkpoint
        torch.save({
            "epoch": epoch,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "scheduler_state_dict": scheduler.state_dict(),
        }, checkpoint_path)

        scheduler.step(val_loss)
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered.")
            break

train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, model_save_path, checkpoint_path)


Epoch 1/20: Training...


Epoch 1/20: Validating...


Epoch 1/20, Train Loss: 1.5747, Val Loss: 1.1964, Val Accuracy: 56.73%
Model saved at epoch 1 with Val Accuracy: 56.73%

Epoch 2/20: Training...


KeyboardInterrupt: 

In [ ]:
# Testing with Per-Class Accuracy
def test_model(model, test_loader, emotion_classes, model_save_path):
    model.load_state_dict(torch.load(model_save_path))
    model.eval()
    correct = [0] * len(emotion_classes)
    total = [0] * len(emotion_classes)
    print("Testing the model...")
    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Testing Progress"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            for i in range(len(labels)):
                label = labels[i].item()
                total[label] += 1
                correct[label] += (predicted[i] == label).item()

    for i, emotion in enumerate(emotion_classes):
        accuracy = 100 * correct[i] / total[i] if total[i] > 0 else 0
        print(f"{emotion}: {accuracy:.2f}%")
    overall_accuracy = sum(correct) / sum(total)
    print(f"Overall Test Accuracy: {overall_accuracy * 100:.2f}%")

test_model(model, test_loader, emotion_classes, model_save_path)